## Evoluční generování strategie pro iterované vězňovo dilema

Pro jednoduchost budeme zatím uvažovat, že agent bere v úvahu jen poslední tah obou hráčů, tj. má omezenou historii interakcí.

In [8]:
import random
from deap import base, creator, tools, algorithms

Kódování vstupu/výstupu:
- 0 reprezentuje "spolupráci"
- 1 reprezentuje "zradu"

Tabulku můžeme reprezentovat jako pole 4 hodnot (0 nebo 1). Index v poli odpovídá kombinaci vstupů:
- Index 0: (0, 0) - Já spolupracuji, protihráč spolupracuje
- Index 1: (0, 1) - Já spolupracuji, protihráč zrazuje
- Index 2: (1, 0) - Já zrazuji, protihráč spolupracuje
- Index 3: (1, 1) - Já zrazuji, protihráč zrazuje

In [9]:
# 1. Kódování tabulky a funkce zrada
def zrada(moje_historie, protihracova_historie, genom):
    """
    Vrátí další tah (0 nebo 1) na základě historie a genomu.

    Args:
        moje_historie: List mých předchozích tahů.
        protihracova_historie: List tahů protihráče.
        genom: Genom strategie (list 4 prvků 0/1).

    Returns:
        0 (spolupráce) nebo 1 (zrada).
    """

    if not moje_historie:
        return 0  # První tah - defaultně spolupracuj

    posledni_muj_tah = moje_historie[-1]
    posledni_protihracuv_tah = protihracova_historie[-1]
    index = posledni_muj_tah * 2 + posledni_protihracuv_tah  # Vypočítáme index do tabulky
    return genom[index]


Simulace hry proti několika různým strategiím (např. vždy spolupracuj, vždy zraď, náhodná strategie, oko za oko). Fitness jedince (genomu) bude celkový zisk bodů ve všech těchto hrách. Vyšší zisk = vyšší fitness.

In [10]:

# 2. Evoluční algoritmus
def evaluate(individual):
    """
    Fitness funkce - simuluje hru proti různým strategiím a vrací skóre.
    """

    celkove_skore = 0

    # Definice protihráčů
    protihraci = {
        "vždy_spolupracuj": [0] * 20,
        "vždy_zraď": [1] * 20,
        "nahodny": [random.randint(0, 1) for _ in range(20)],
        "oko_za_oko": []  # Bude dopočítáno
    }

    # Simulace her
    for strategie, tahy_protihrace in protihraci.items():
        moje_tahy = []
        if strategie == "oko_za_oko":
          tahy_protihrace = [0] # Začíná spoluprací
        
        for kolo in range(len(tahy_protihrace)):
            muj_tah = zrada(moje_tahy, tahy_protihrace, individual)
            moje_tahy.append(muj_tah)

            if strategie == "oko_za_oko":
              if moje_tahy:
                tah_protihrace = tahy_protihrace[-1] # Opakuje poslední můj tah
              else:
                tah_protihrace = 0 # Začíná spoluprací
              tahy_protihrace.append(tah_protihrace)

        # Výpočet skóre (příklad)
        for i in range(len(moje_tahy)):
            if moje_tahy[i] == 0 and tahy_protihrace[i] == 0:
                celkove_skore += 3  # Odměna za spolupráci
            elif moje_tahy[i] == 0 and tahy_protihrace[i] == 1:
                celkove_skore += 0  # Jsem pako
            elif moje_tahy[i] == 1 and tahy_protihrace[i] == 0:
                celkove_skore += 5  # Jsem zrádce
            else:
                celkove_skore += 1  # Oba zrazujeme

    return (celkove_skore,)

Nastavení DEAP: definice fitness (maximalizace), definice jedince (genom = list 4 bitů)

In [11]:
# DEAP setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=4)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

Spuštění evolučního algoritmu a výpis nejlepší strategie:

In [12]:
if __name__ == '__main__':
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("avg", sum)
    stats.register("max", max)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=30, stats=stats, halloffame=hof, verbose=True)

    # Nejlepší strategie
    best_strategie = hof[0]
    print("Nejlepší strategie (genom):", best_strategie)

gen	nevals	avg  	max
0  	100   	12672	191
1  	55    	14443	191
2  	65    	16218	191
3  	50    	17345	202
4  	61    	17610	202
5  	77    	17704	202
6  	50    	17851	198
7  	67    	18052	198
8  	66    	17974	203
9  	56    	18135	202
10 	63    	18241	199
11 	57    	18092	206
12 	69    	18335	199
13 	61    	17967	198
14 	60    	17978	202
15 	58    	18183	199
16 	55    	18349	198
17 	76    	18055	198
18 	56    	18184	202
19 	67    	17985	198
20 	59    	18127	198
21 	68    	17891	206
22 	57    	18229	206
23 	56    	17972	206
24 	60    	18041	206
25 	71    	17903	206
26 	58    	18165	206
27 	51    	18352	206
28 	62    	18272	206
29 	63    	18316	206
30 	65    	18239	206
Nejlepší strategie (genom): [1, 1, 1, 1]


Ukázka použití funkce zrada:

In [13]:
moje_historie = []
protihracova_historie = []
for _ in range(10):
    muj_tah = zrada(moje_historie, protihracova_historie, best_strategie)
    protihracuv_tah = random.randint(0, 1)  # Náhodný protihráč (pro ukázku)
    moje_historie.append(muj_tah)
    protihracova_historie.append(protihracuv_tah)
    print(f"Můj tah: {muj_tah}, Tah protihráče: {protihracuv_tah}")

Můj tah: 0, Tah protihráče: 1
Můj tah: 1, Tah protihráče: 1
Můj tah: 1, Tah protihráče: 0
Můj tah: 1, Tah protihráče: 0
Můj tah: 1, Tah protihráče: 1
Můj tah: 1, Tah protihráče: 0
Můj tah: 1, Tah protihráče: 1
Můj tah: 1, Tah protihráče: 1
Můj tah: 1, Tah protihráče: 1
Můj tah: 1, Tah protihráče: 0


#### Závěr
Jedná se o jednoduchou základní implementaci, vhodné by bylo použití sofistikovanějších protihráčů, příp. i delší historie tahů. Stejně tak je pro vývoj strategií klíčová matice výpočtu odměn (skóre). Pochopitelně i nastavení parametrů evolučního algoritmu.